# Hadamard Test Tutorial

This tutorial will guide you through the understanding and implementation of the Hadamard Test in quantum computing. We'll explore why the Hadamard Test is important, delve into its applications, and provide Python code snippets for practical understanding.

## Introduction to the Hadamard Test

The Hadamard Test is a significant quantum algorithm used to measure the real and imaginary parts of the expectation value of a unitary operator. This is pivotal in quantum computing for various applications, including quantum algorithm validation, phase estimation, and more.

### Why is the Hadamard Test Important?

The Hadamard Test is crucial for several reasons:

1. **Phase Estimation:** It helps in estimating the phase (angle) that a quantum state acquires after a unitary transformation, which is vital for algorithms like Shor's algorithm.
2. **Quantum Algorithm Validation:** It provides a way to verify the correctness of quantum algorithms by measuring their output states.
3. **General Quantum Measurements:** The test enables the measurement of complex quantities that are otherwise difficult to obtain directly using standard quantum computational methods.

## Applications of the Hadamard Test

The Hadamard Test finds applications in various fields of quantum computing:

- **Quantum Algorithm Development:** It's used in the development and debugging of quantum algorithms.
- **Quantum Chemistry:** In simulating molecular systems, it helps in measuring the expectation values of Hamiltonians.
- **Quantum Machine Learning:** It's used in quantum machine learning algorithms to estimate gradients and other quantities.

---

## Experimenting with the Hadamard Test



In [ ]:
import classiq
classiq.authenticate()

In [ ]:
from classiq import *
import numpy as np

## Implementing Hadamard Test


### Objective
Find the expectation value of $\langle\psi|U|\psi\rangle$, using a quantum function `hadamard_test_r`. Find the expectation value of $\langle\psi|U\psi\rangle$'s imaginary part using `hadamard_test_i`.

### Process Overview

#### Step 1: Initialize $|\psi\rangle$

- **Random State Creation**: Generate a random amplitude for $|\psi\rangle$.
- **Quantum State Preparation**: Utilize `prepare_amplitudes` from the Classiq SDK to initialize the quantum state of the $\psi$ qubit based on the amplitude.

### Implementation Details

Generate an array of random numbers within the range [-1, 1]. The array size equals the number of possible superposition states for `NUM_QUBITS`, aligning with the quantum system's complexity. Since we are only using one qubit, we will generate an array of size two ($2^1$).

In [ ]:
np.random.seed(18)

NUM_QUBITS = 1
amps1 = 1 - 2 * np.random.rand(2**NUM_QUBITS)
amps1 = amps1 / np.linalg.norm(amps1)
print(amps1)

#### Step 2: Create a Quantum Function for the Real Expectation Value
The steps for a hadamard test are as follows:
- Apply a hadamard to the ancilla qubit
- Apply a controlled unitary operation with the ancilla qubit as the control and $|\psi\rangle$ as the target. In our case, the controlled unitary operation is the controlled-not(CX) gate.
- Apply a hadamard to the ancilla qubit
- Measure the ancilla 

Notice how `hadamard_test_r` implements the steps above in readable and concise manner.

In [ ]:
@QFunc
def hadamard_test_r(psi: QBit, ancilla:Output[QBit]):
    allocate(out=ancilla,num_qubits=1)
    hadamard_transform(ancilla)
    CX(ancilla,psi)
    hadamard_transform(ancilla)

In [ ]:
@QFunc
def main(ancilla_real:Output[QBit]):
    psi = QBit("state1")
    prepare_amplitudes(amps1.tolist(), 0.0, psi)
    hadamard_test_r(psi, ancilla_real)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

With the quantum program created, we can now calculate the expectation value using the following formula:
$E=P(0)−P(1)$

Where $P(0)$ and $P(1)$ are the probabilities of measuring the qubit in state ∣0 and ∣1⟩, respectively. These probabilities can be computed from the counts dictionary as follows:

$P(0) = \frac{Counts(0)}{Total Counts}$
<br><br>

$P(1) = \frac{Counts(1)}{Total Counts}$


In [ ]:
result = execute(qprog).result()

p0 = result[0].value.counts["0"] / sum(result[0].value.counts.values())
p1 = result[0].value.counts["1"] / sum(result[0].value.counts.values())
expectation = p0 - p1

print(f"Expectation value from real part: {expectation}")

#### Step 2: Create a Quantum Function for the Imaginary Expectation Value
The steps for a hadamard test are as follows:
- Apply a hadamard to the ancilla qubit
- Apply a phase shift of $\frac{\pi}{2}$ to the ancilla qubit
- Apply a controlled unitary operation with the ancilla qubit as the control and $|\psi\rangle$ as the target. In our case, the controlled unitary operation is the controlled-not(CX) gate.
- Apply a hadamard to the ancilla qubit
- Measure the ancilla 

Notice how `hadamard_test_i` implements the steps above in a readable and concise manner.

In [ ]:
@QFunc
def hadamard_test_i(psi: QBit, ancilla:Output[QBit]):
    allocate(out=ancilla,num_qubits=1)
    hadamard_transform(ancilla)
    RZ(np.pi/2,ancilla)
    CX(ancilla,psi)
    hadamard_transform(ancilla)

### Step 3: Putting it All Together
- Prepare $|\psi\rangle$.
- Use use `hadamard_test_i` to find the expectation value for imaginary parts.

In [ ]:
@QFunc
def main(ancilla_imag:Output[QBit]):
    psi = QBit("state1")
    prepare_amplitudes(amps1.tolist(), 0.0, psi)
    hadamard_test_i(psi, ancilla_imag)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Once again we will calculate the expectation value using the following formula:
<br><br>
$E=P(0)−P(1)$


In [ ]:
result = execute(qprog).result()

p0 = result[0].value.counts["0"] / sum(result[0].value.counts.values())
p1 = result[0].value.counts["1"] / sum(result[0].value.counts.values())
expectation = p0 - p1

print(f"Expectation value from imaginary part: {expectation}")

### Example 2: 

We wil now use the Hadamard-Test to create the notorious Swap-Test. Swap-Test compares two quantum states, $|\psi\rangle$ and $|\phi\rangle$. The Swap-Test is the same as the Hadamard test for real parts, except it uses a bit-wise controlled Swap Gate as the controlled unitary operator. For Swap-Test the control qubit will be the ancilla and the targets will be $\psi$ and $\phi$.

In [ ]:
@QFunc
def bit_wise_swap(psi: QArray[QBit], phi: QArray[QBit]):
    repeat(psi.len(), lambda i: SWAP(psi[i],phi[i]))

@QFunc
def local_swap_test(ancilla:Output[QBit], psi: QArray[QBit], phi: QArray[QBit]):
    allocate(1,ancilla)
    H(ancilla)
    control(lambda: bit_wise_swap(psi,phi),ancilla)
    H(ancilla)

Now we generate two random arrays to initialize psi and phi. This will provide us will two different states to compare using our local swap test. Notice how difficult implementing swap test at a gate level is, thankfully we have Classiq's built-in `swap_test` function.

In [ ]:
@QFunc
def main(ancilla:Output[QBit]):
    psi_state = [0,0.333,0.333,0.334]
    phi_state = [0.334,0.333,0.333,0]

    psi = QArray('psi')
    phi = QArray('phi')
    prepare_state(probabilities=psi_state, bound=1e-4, out=psi)
    prepare_state(probabilities=phi_state, bound=1e-4, out=phi)
    local_swap_test(ancilla,psi, phi)

qmod = create_model(main)

qprog = synthesize(qmod)
show(qprog)

In [ ]:
result = execute(qprog).result()

state_overlap = np.sqrt(
    2 * result[0].value.counts["0"] / sum(result[0].value.counts.values()) - 1
)

print(f"States overlap from Swap-Test result: {state_overlap}")



---

## Conclusion

The Hadamard Test is a foundational tool in quantum computing for measuring the expectation values of unitary operators. Through this tutorial, you've learned its importance, applications, and implementation using Qiskit. Experimenting with different unitary operators and quantum circuits can deepen your understanding of quantum computing principles.

### The Classiq Platform: Empowering Quantum Innovation

The Classiq platform has emerged as an indispensable tool in the quantum computing industry. Its strength and importance lie in demystifying quantum programming, offering an accessible yet powerful avenue for users to design, prototype, and analyze quantum algorithms with ease. Classiq's user-friendly interface and advanced features bridge the gap between complex quantum theories and practical applications, enabling both beginners and seasoned experts to unlock quantum computing's vast potential. Through facilitating the design of sophisticated quantum functions, as demonstrated with our quadratic examples, Classiq accelerates the development of quantum applications, positioning itself as a catalyst for breakthroughs in computational power and innovation.

---

## Thank You!
#### Make sure to checkout the classiq platform here: https://platform.classiq.io